## <center>회귀분석2 HW 2</center>

### **CH06, 07**

### 1. 어떤 공장에서 물의 소비량을 조사하기 위하여 매달의 물소비량($y$), 평균온도($x_1$), 작업일수($x_2$)와 작업량($x_3$)에 관한 데이터를 얻었다.

In [52]:
library(dplyr)

In [1]:
df = data.frame(y = c(2.8, 3.9, 3.9, 4.4, 3.1, 3.1, 3.5, 3.6, 3.0, 3.3),
                x1 = c(10, 24, 25, 28, 15, 18, 22, 22, 12, 15),
                x2 = c(27, 26, 28, 26, 30, 24, 27, 25, 27, 25),
                x3 = c(64, 72, 80, 88, 81, 45, 46, 69, 54, 39))
df

y,x1,x2,x3
<dbl>,<dbl>,<dbl>,<dbl>
2.8,10,27,64
3.9,24,26,72
3.9,25,28,80
4.4,28,26,88
3.1,15,30,81
3.1,18,24,45
3.5,22,27,46
3.6,22,25,69
3.0,12,27,54


**(1) 데이터로부터 회귀모형, $\hat{y} = \hat{\beta_0} + \hat{\beta_1}x_1 + \hat{\beta_2}x_2 + \hat{\beta_3}x_3$을 구하여라.**

In [3]:
model <- lm(y~., df)
model


Call:
lm(formula = y ~ ., data = df)

Coefficients:
(Intercept)           x1           x2           x3  
   2.409213     0.069788    -0.024767     0.005864  


적합된 회귀모형 : $\hat{y} = 2.4092 + 0.0698x_1 - 0.0248x_2 + 0.0059x_3$

**(2) $\hat{\beta_1},~\hat{\beta_2},~\hat{\beta_3}$ 의 의미는 무엇인가?**

* $\hat{\beta_1}$ : 작업일수($x_2$)와 작업량($x_3$)이 고정되어 있을 때 평균온도를 1만큼 상승시키면 물 소비량이 0.0698만큼 증가한다.
* $\hat{\beta_2}$ : 평균온도($x_1$)와 작업량($x_3$)이 고정되어 있을 때, 작업일수가 하루 늘어나면 물 소비량이 0.0248만큼 감소한다. (?)
* $\hat{\beta_3}$ : 평균온도($x_1$)와 작업일수($x_2$)가 고정되어 있을 때, 작업량이 1만큼 늘어나면 물 소비량은 0.0059만큼 증가한다.

**(3) 분산분석표를 작성하고, 결정계수 $R^2$를 구하시오.**

`-` 분산분석표 작성

In [55]:
anova(model)

,Df,Sum Sq,Mean Sq,F value,Pr(>F)
,<int>,<dbl>,<dbl>,<dbl>,<dbl>
x1,1,2.004315887,2.004315887,67.73858599,0.0001737553
x2,1,0.002273071,0.002273071,0.07682153,0.7909535228
x3,1,0.039877142,0.039877142,1.34770234,0.2897755827
Residuals,6,0.177533900,0.029588983,NA,NA


In [63]:
SSR <- apply(anova(model)[1:3, 1:2], 2, sum)
SST <- SSR + anova(model)[4, 1:2]

In [74]:
anov = rbind(SSR, anova(model)[4, 1:2], SST)
rownames(anov) <- c("회귀", "잔차", "총")
anov = cbind(anov, (anov[2] / anov[1]))
anov = cbind(anov, anov[1,3] / anov[2,3], pf(anov[1,3] / anov[2,3], 3, 6, lower.tail = FALSE))
colnames(anov) <- c("Df", "Sum Sq", "Mean Sq", "F value", "Pr(>F)")
anov[3, 3:5] <- NA
anov[2, 4:5] <- NA
anov

,Df,Sum Sq,Mean Sq,F value,Pr(>F)
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
회귀,3,2.0464661,0.68215537,23.05437,0.001078865
잔차,6,0.1775339,0.02958898,NA,NA
총,9,2.2240000,NA,NA,NA


`-` $R^2$ 값 계산

In [76]:
summary(model)$r.squared

[1] 0.9201736

**(4) 수정된 결정계수를 구하여라.**

In [77]:
summary(model)$adj.r.squared

[1] 0.8802604

**(5) $Var(\hat{\beta_3})$ 을 구하시오. $\sigma^2$을 MSE로 추정하면 $\widehat{Var}(\hat{\beta_3})$ 은 무엇인가?**

In [105]:
X <- cbind(1, as.matrix(df[2:4]))
MSE <- (summary(model)$sigma)^2
solve_XX <- solve(t(X)%*%X)
solve_XX

,,x1,x2,x3
,42.8460778,-0.274517258,-1.666784957,0.1044984818
x1,-0.2745173,0.005399764,0.009802163,-0.0013851965
x2,-1.6667850,0.009802163,0.067921641,-0.0050213139
x3,0.1044985,-0.001385196,-0.005021314,0.0008624387


`-` $Var(\hat{\beta_3})$ 의 값

In [93]:
print(paste(solve_XX[4,4], "× Var(y)"))

[1] "0.0008624387021274 × Var(y)"


`-` $\widehat{Var}(\hat{\beta_3})$ 의 값

In [102]:
solve_XX[4,4]*MSE

[1] 2.551868e-05

In [104]:
summary(model)$coef[4,2]^2

[1] 2.551868e-05

**(6) $x_1 = 20,~x_2 = 27,~x_3 = 60$에서 평균 물소비량을 추정하시오. 이 추정된 소비량의 표준편차의 추정값을 구하시오.**

`-` 평균 물 소비량 추정 : $\hat{\mu_0} = \widehat{E(Y | \textbf{x})}$

In [124]:
predict(model, newdata = data.frame(x1 = 20, x2 = 27, x3 = 60))

1 
3.488141

`-` 추정된 평균 물 소비량의 표준편차 추정 : $\widehat{Var(\hat{\mu_0})}$

In [122]:
x_0 <- c(1, 20, 27, 60)

In [121]:
as.numeric(t(x_0)%*%solve_XX%*%x_0*MSE)

[1] 0.005065205

**(7) $x_1 = 20,~x_2 = 27,~x_3 = 60$ 에서 어느 한 달의 물 소비량 $y_s$를 예측하여라. 이 예측된 물 소비량의 표준편차의 추정값을 구하시오.**

`-` 개별 물 소비량 추정 : $\hat{y_s} = \hat{\mu_0}$

In [141]:
predict(model, newdata = data.frame(x1 = 20, x2 = 27, x3 = 60))

1 
3.488141

`-` 추정된 개별 물 소비량의 표준편차 추정 : $\widehat{Var(\hat{y_s}})$

In [125]:
as.numeric((1 + t(x_0)%*%solve_XX%*%x_0)*MSE)

[1] 0.03465419

**(8) $\beta_3$ 의 $95\%$ 신뢰구간을 구하시오. 이 신뢰구간의 의미를 해석하시오.**

In [133]:
summary(model)$coef

,Estimate,Std. Error,t value,Pr(>|t|)
(Intercept),2.409213010,1.125953765,2.1397087,0.076181015
x1,0.069788005,0.012640155,5.5211353,0.001485354
x2,-0.024766597,0.044830038,-0.5524554,0.600595338
x3,0.005864434,0.005051602,1.1609058,0.289775583


In [140]:
sigma_3 <- summary(model)$coef[4,2]
lwb <- model$coef[4] - qt(0.975, 10-4)*sigma_3
upb <- model$coef[4] + qt(0.975, 10-4)*sigma_3

print(paste("회귀계수에 대한 95% CI : (", lwb, ", ", upb, ")", sep = ""))

[1] "회귀계수에 대한 95% CI : (-0.00649639079638348, 0.0182252595602561)"


> 해당 신뢰구간이 $\beta_3$ 를 포함할 확률이 95%라는 의미이다. 개별 회귀계수의 검정에서 귀무가설이 $H_0 : \beta_3 = 0$ 인데, 0이 이 구간에 포함되어 있다. 따라서 해당 신뢰구간은 회귀계수가 유의하지 않다는 것을 뒷받침한다.

**(9) $\beta_1$ 의 $99\%$ 신뢰구간을 구하시오. 이 신뢰구간의 의미를 해석하시오.**

In [142]:
sigma_1 <- summary(model)$coef[2,2]
lwb <- model$coef[2] - qt(0.975, 10-4)*sigma_3
upb <- model$coef[2] + qt(0.975, 10-4)*sigma_3

print(paste("회귀계수에 대한 95% CI : (", lwb, ", ", upb, ")", sep = ""))

[1] "회귀계수에 대한 95% CI : (0.057427180184659, 0.0821488305412985)"


> 해당 신뢰구간이 $\beta_1$ 를 포함할 확률이 99%라는 의미이다. 개별 회귀계수의 검정에서 귀무가설이 $H_0 : \beta_1 = 0$ 인데, 0이 이 구간에 포함되어 있지 않으므로, 해당 신뢰구간은 회귀계수가 유의하다는 것을 뒷받침한다.

**(10) $x_1 = 20,~x_2 = 27,~x_3 = 60$ 에서 평균 물 소비량, $E(y)$ 의 $95\%$ 신뢰구간을 구하시오.**

In [156]:
CI <- predict(model, newdata = data.frame(x1 = 20, x2 = 27, x3 = 60), interval = "confidence", level = 0.95)
print(paste("평균 물 소비량의 95% CI : (", CI[2], ", ", CI[3], ")", sep = ""))

[1] "평균 물 소비량의 95% CI : (3.31399371197438, 3.66228839915852)"


**(11) 가설 $H_0 : \beta_1 = 0,~ vs. ~ H_1 : \beta_1 > 0$ 을 $\alpha = 0.05$ 로 검정하시오.**

ⅰ) 검정통계량 $T = \frac{\hat{\beta_1}-0}{\sigma_{\hat{\beta_1}}} \sim N(\beta_1, \sigma_{\hat{\beta_1}}^2)$

ⅱ) 검정통계량의 관측값

In [164]:
t_value = model$coef[2] / summary(model)$coef[2,2]
t_value

x1 
5.521135

ⅲ) p-value

In [172]:
p_value = pt(t_value, 10-4, lower.tail = FALSE)
p_value

x1 
0.0007426772

In [171]:
summary(model)$coef[2,4]/2

[1] 0.0007426772

In [173]:
p_value < 0.05

x1 
TRUE

> p-value가 유의수준 $\alpha = 0.05$ 보다 작으므로, 귀무가설을 기각하고 대립가설을 수용한다. 즉, $\beta_1$ 은 0보다 크다.

**(12) 가설 $H_0 : \beta_1 = \beta_2 = \beta_3$ 을 $\alpha = 0.05$ 로 검정하시오.**